In [2]:
import torch
import numpy as np
from botorch.models import SingleTaskGP
from botorch.test_functions import Ackley,Beale,Branin,Rosenbrock,SixHumpCamel,Hartmann,Powell,DixonPrice,Levy,StyblinskiTang,Griewank
from gpytorch.mlls import ExactMarginalLogLikelihood
from botorch.fit import fit_gpytorch_mll
from botorch.optim import optimize_acqf
from botorch.utils.transforms import unnormalize,normalize
from torch.quasirandom import SobolEngine
from botorch.test_functions import Ackley,Beale,Branin,Rosenbrock,SixHumpCamel,Hartmann,Rastrigin,Powell,DixonPrice,Levy,StyblinskiTang,Griewank



from model import DerivativeExactGPSEModel,DerivativeExactGPSEModel_2
# from acquisition import (MES_KnownOptimum,TruncatedExpectedImprovement,ExpectedImprovement,Fstar_pdf,
#                             Fstar_pdf_GradientEnhanced,Fstar_pdf_GradientEnhanced_fantasy,
#                             TruncatedExpectedImprovement_GradientEnhanced_fantasy,
#                             TruncatedExpectedImprovement_GradientEnhanced_fantasy_2,
#                             TruncatedExpectedImprovement_GradientEnhanced_fantasy_parallel)
from acquisition import  My_acquisition_opt


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
dtype = torch.double

torch.set_default_dtype(dtype)



def get_initial_points(dim, n_pts, seed=0):
    sobol = SobolEngine(dimension=dim, scramble=True, seed=seed)
    X_init = sobol.draw(n=n_pts).to(dtype=dtype, device=device)
    return X_init

In [22]:
#!/usr/bin/env python3
# Copyright (c) Meta Platforms, Inc. and affiliates.
#
# This source code is licensed under the MIT license found in the
# LICENSE file in the root directory of this source tree.

r"""
Analytic Acquisition Functions that evaluate the posterior without performing
Monte-Carlo sampling.
"""

from __future__ import annotations

import math

from abc import ABC

from contextlib import nullcontext
from copy import deepcopy

from typing import Dict, Optional, Tuple, Union

import torch
from botorch.acquisition.acquisition import AcquisitionFunction
from botorch.acquisition.objective import PosteriorTransform
from botorch.exceptions import UnsupportedError
from botorch.models.gp_regression import FixedNoiseGP
from botorch.models.gpytorch import GPyTorchModel
from botorch.models.model import Model
from botorch.utils.constants import get_constants_like
from botorch.utils.probability import MVNXPB
from botorch.utils.probability.utils import (
    log_ndtr as log_Phi,
    log_phi,
    log_prob_normal_in,
    ndtr as Phi,
    phi,
)
from botorch.utils.safe_math import log1mexp, logmeanexp
from botorch.utils.transforms import convert_to_target_pre_hook, t_batch_mode_transform
from torch import Tensor
from torch.nn.functional import pad

_sqrt_2pi = math.sqrt(2 * math.pi)
# the following two numbers are needed for _log_ei_helper
_neg_inv_sqrt2 = -(2**-0.5)
_log_sqrt_pi_div_2 = math.log(math.pi / 2) / 2


from botorch.acquisition import AnalyticAcquisitionFunction
from botorch.models.model import Model
from typing import Any, Optional, Union
from torch import Tensor
from botorch.acquisition.objective import PosteriorTransform
from botorch.utils.transforms import convert_to_target_pre_hook, t_batch_mode_transform
# from botorch.acquisition import *
from abc import ABC
from typing import Dict, Optional, Tuple, Union
import botorch.acquisition as temp
import botorch.acquisition 
from botorch.utils.probability.utils import log_phi, log_ndtr,ndtr as Phi, phi

# def _gamma(
#     mean: Tensor, sigma: Tensor, fstar: Tensor, maximize: bool
# ) -> Tensor:
#     """Returns `u = (mean - best_f) / sigma`, -u if maximize == True."""
#     u = (fstar - mean) / sigma
#     return u if maximize else -u



class AnalyticAcquisitionFunction(AcquisitionFunction, ABC):
    r"""
    Base class for analytic acquisition functions.

    :meta private:
    """

    def __init__(
        self,
        model: Model,
        posterior_transform: Optional[PosteriorTransform] = None,
        **kwargs,
    ) -> None:
        r"""Base constructor for analytic acquisition functions.

        Args:
            model: A fitted single-outcome model.
            posterior_transform: A PosteriorTransform. If using a multi-output model,
                a PosteriorTransform that transforms the multi-output posterior into a
                single-output posterior is required.
        """
        super().__init__(model=model)
        posterior_transform = self._deprecate_acqf_objective(
            posterior_transform=posterior_transform,
            objective=kwargs.get("objective"),
        )
        if posterior_transform is None:
            if model.num_outputs != 1:
                raise UnsupportedError(
                    "Must specify a posterior transform when using a "
                    "multi-output model."
                )
        else:
            if not isinstance(posterior_transform, PosteriorTransform):
                raise UnsupportedError(
                    "AnalyticAcquisitionFunctions only support PosteriorTransforms."
                )
        self.posterior_transform = posterior_transform

    def set_X_pending(self, X_pending: Optional[Tensor] = None) -> None:
        raise UnsupportedError(
            "Analytic acquisition functions do not account for X_pending yet."
        )

    def _mean_and_sigma(
        self, X: Tensor, compute_sigma: bool = True, min_var: float = 1e-12
    ) -> Tuple[Tensor, Optional[Tensor]]:
        """Computes the first and second moments of the model posterior.

        Args:
            X: `batch_shape x q x d`-dim Tensor of model inputs.
            compute_sigma: Boolean indicating whether or not to compute the second
                moment (default: True).
            min_var: The minimum value the variance is clamped too. Should be positive.

        Returns:
            A tuple of tensors containing the first and second moments of the model
            posterior. Removes the last two dimensions if they have size one. Only
            returns a single tensor of means if compute_sigma is True.
        """
        self.to(device=X.device)  # ensures buffers / parameters are on the same device
        posterior = self.model.posterior(
            X=X, posterior_transform=self.posterior_transform
        )
        mean = posterior.mean.squeeze(-2).squeeze(-1)  # removing redundant dimensions
        if not compute_sigma:
            return mean, None
        sigma = posterior.variance.clamp_min(min_var).sqrt().view(mean.shape)
        return mean, sigma


# --------------- Helper functions for analytic acquisition functions. ---------------


def _scaled_improvement(
    mean: Tensor, sigma: Tensor, best_f: Tensor, maximize: bool
) -> Tensor:
    """Returns `u = (mean - best_f) / sigma`, -u if maximize == True."""
    u = (mean - best_f) / sigma
    return u if maximize else -u


def _gamma(
    mean: Tensor, sigma: Tensor, fstar: Tensor, maximize: bool
) -> Tensor:
    """Returns `u = (mean - best_f) / sigma`, -u if maximize == True."""
    u = (fstar - mean) / sigma
    return u if maximize else -u


def _ei_helper(u: Tensor) -> Tensor:
    """Computes phi(u) + u * Phi(u), where phi and Phi are the standard normal
    pdf and cdf, respectively. This is used to compute Expected Improvement.
    """
    return phi(u) + u * Phi(u)



from torch.nn.functional import softplus
import math

from typing import Callable, Tuple, Union

def cauchy(x: Tensor) -> Tensor:
    """Computes a Lorentzian, i.e. an un-normalized Cauchy density function."""
    return 1 / (1 + x.square())

def fatplus(x: Tensor, tau: Union[float, Tensor] = 0.5) -> Tensor:
    """Computes a fat-tailed approximation to `ReLU(x) = max(x, 0)` by linearly
    combining a regular softplus function and the density function of a Cauchy
    distribution. The coefficient `alpha` of the Cauchy density is chosen to guarantee
    monotonicity and convexity.

    Args:
        x: A Tensor on whose values to compute the smoothed function.
        tau: Temperature parameter controlling the smoothness of the approximation.

    Returns:
        A Tensor of values of the fat-tailed softplus.
    """

    def _fatplus(x: Tensor) -> Tensor:
        alpha = 1e-1  # guarantees monotonicity and convexity (TODO: ref + Lemma 4)
        return softplus(x) + alpha * cauchy(x)

    return tau * _fatplus(x / tau)


#########################################################################

import concurrent.futures
from functools import partial
import numpy as np 
# Define a function to be executed in parallel
def logpdf_calculation(x_temp,model,D,fstar):
        
            
            x_temp = x_temp.reshape(1,1,D)
            
            model_temp = model.get_fantasy_model(x_temp.reshape(-1,D), torch.tensor([fstar.item()]).reshape(-1,1))
            model_temp.N = model.N+1
            model_temp.train_targets = model_temp.train_targets.reshape(model_temp.N)
            
            mean_d, variance_d = model_temp.posterior_derivative(x_temp.reshape(-1,1,D))
               
            variance_d_new = torch.diagonal(variance_d, dim1=-2, dim2=-1)
            sigma_d_new = variance_d_new.sqrt()

            u = (torch.tensor(0.)-mean_d)/sigma_d_new
            
            max_dis = 20.
            u = fatplus(u+max_dis,tau=0.2)-torch.as_tensor(max_dis) 
            u = - (fatplus(-u+max_dis,tau=0.2)-torch.as_tensor(max_dis))

            logpdf_temp = torch.sum(log_phi(u),dim=1).item()
            
                
            return logpdf_temp





class TruncatedExpectedImprovement_GradientEnhanced_fantasy_parallel(AnalyticAcquisitionFunction):
   

    def __init__(
        self,
        model: Model,
        best_f: Union[float, Tensor],
        fstar: Union[float, Tensor],
        posterior_transform: Optional[PosteriorTransform] = None,
        maximize: bool = True,
        **kwargs,
    ):
        
        super().__init__(model=model, posterior_transform=posterior_transform, **kwargs)
        self.register_buffer("best_f", torch.as_tensor(best_f))
        self.register_buffer("fstar", torch.as_tensor(fstar))
        self.maximize = maximize

    @t_batch_mode_transform(expected_q=1)
    def forward(self, X: Tensor) -> Tensor:
       
        mean, sigma = self._mean_and_sigma(X)
        u1 = _scaled_improvement(mean, sigma, self.best_f, self.maximize)
        u2 = _scaled_improvement(mean, sigma, self.fstar, self.maximize)
        
        max_dis = 20.
        u1 = fatplus(u1+max_dis,tau=0.2)-torch.as_tensor(max_dis) 
        u1 = - (fatplus(-u1+max_dis,tau=0.2)-torch.as_tensor(max_dis))
        
        max_dis = 20.
        u2 = fatplus(u2+max_dis,tau=0.2)-torch.as_tensor(max_dis) 
        u2 = - (fatplus(-u2+max_dis,tau=0.2)-torch.as_tensor(max_dis))
        
        part1 = ((sigma * _ei_helper(u1)) -(sigma * _ei_helper(u2))).log()
        
        # part 2 calculation
        D = X.shape[-1]
        partial_logpdf_calculation = partial(logpdf_calculation, model=self.model,D=D,fstar=self.fstar)
         
        X_temp_list = [X[i, :, :] for i in range(X.size(0))]
        
        with concurrent.futures.ProcessPoolExecutor() as executor:
            # Map the new function to the data using multiple processes
            results = executor.map(partial_logpdf_calculation, X_temp_list)
        
 
        # logpdf_total = torch.stack(list(results))
     
        # part2 = logpdf_total

        return results


In [23]:
iter_num = 10
N = 1
function = Branin(negate=True)
fstar = -0.397887 
acquisition = 'TruncatedExpectedImprovement_GradientEnhanced_fantasy_parallel'

In [24]:
bounds=function.bounds.to(device)
dim = bounds.shape[1]
standard_bounds=torch.tensor([0.,1.]*dim).reshape(-1,2).T.to(device)


record = []

for exp in range(N):
    

    print(exp)
    torch.manual_seed(exp)

    train_x_standard = get_initial_points(dim,4*dim,exp).to(device)
    train_x = unnormalize(train_x_standard, bounds).reshape(-1,dim)
    train_obj = function(train_x).unsqueeze(-1)

    best_value = train_obj.max().item()
    best_value_holder = [best_value]
    
    print(best_value_holder[-1])

    for i in range (1):

        train_x_standard = normalize(train_x, bounds).to(device)
        train_obj_standard = (train_obj - train_obj.mean()) / train_obj.std()
        fstar_standard = (fstar - train_obj.mean()) / train_obj.std()
        
        torch.manual_seed(exp+iter_num)
        model = DerivativeExactGPSEModel(dim,train_x_standard, train_obj_standard).to(device)    

        mll = ExactMarginalLogLikelihood(model.likelihood, model) .to(device)

        torch.manual_seed(exp+iter_num)
        
        try:
            fit_gpytorch_mll(mll)
        except:
            pass
        
        
        torch.manual_seed(exp+iter_num)

        if acquisition == 'TruncatedExpectedImprovement_GradientEnhanced_fantasy_parallel':
            AF = TruncatedExpectedImprovement_GradientEnhanced_fantasy_parallel(model=model, best_f=train_obj_standard.max().item(),fstar=fstar_standard) .to(device)




        
    #     try:
    #         np.random.seed(exp+i)
    #         new_point_analytic = My_acquisition_opt(AF,dim)
    #         new_point_analytic = torch.tensor(new_point_analytic).reshape(-1,dim).to(device)
    #     except:
    #         print('problem occur')
    #         if acquisition == 'TruncatedExpectedImprovement_GradientEnhanced_fantasy' or acquisition == 'TruncatedExpectedImprovement_GradientEnhanced_fantasy_2':
    #             AF_temp = TruncatedExpectedImprovement(model=model, best_f=train_obj_standard.max().item(),fstar=fstar_standard) .to(device)
            
    #         np.random.seed(exp+i)
    #         new_point_analytic = My_acquisition_opt(AF_temp,dim)
    #         new_point_analytic = torch.tensor(new_point_analytic).reshape(-1,dim).to(device)
        
    #     print(new_point_analytic)

    #     next_x = unnormalize(new_point_analytic, bounds).reshape(-1,dim)
    #     new_obj = function(next_x).unsqueeze(-1) .to(device)


    #     train_x = torch.cat((train_x, next_x))
    #     train_obj = torch.cat((train_obj, new_obj))

    #     best_value = train_obj.max().item()
    #     best_value_holder.append(best_value)

    #     print(best_value_holder[-1])

    # best_value_holder = np.array(best_value_holder)
    # record.append(best_value_holder)
    


0
-3.5451956715421797


In [25]:
AF(torch.tensor([0.5,0.5]).reshape(1,1,dim))

AttributeError: 'generator' object has no attribute 'shape'

In [7]:
dim = 2
aa = torch.tensor([[0.5,0.5],[0.3,0.6]]).reshape(2,1,dim)

In [8]:
[aa[i, :, :] for i in range(aa.size(0))]

[tensor([[0.5000, 0.5000]]), tensor([[0.3000, 0.6000]])]

In [41]:
import concurrent.futures
from functools import partial
import numpy as np 
# Define a function to be executed in parallel
def logpdf_calculation(x_temp,model,D,fstar):
        
            
            x_temp = x_temp.reshape(1,1,D)
            
            model_temp = model.get_fantasy_model(x_temp.reshape(-1,D), torch.tensor([fstar.item()]).reshape(-1,1))
            model_temp.N = model.N+1
            model_temp.train_targets = model_temp.train_targets.reshape(model_temp.N)
            
            mean_d, variance_d = model_temp.posterior_derivative(x_temp.reshape(-1,1,D))
               
            variance_d_new = torch.diagonal(variance_d, dim1=-2, dim2=-1)
            sigma_d_new = variance_d_new.sqrt()

            u = (torch.tensor(0.)-mean_d)/sigma_d_new
            
            max_dis = 20.
            u = fatplus(u+max_dis,tau=0.2)-torch.as_tensor(max_dis) 
            u = - (fatplus(-u+max_dis,tau=0.2)-torch.as_tensor(max_dis))

            logpdf_temp = torch.sum(log_phi(u),dim=1).item()
            
                
            return logpdf_temp.detach().numpy()

In [42]:
import torch
import torch.multiprocessing as mp

aa = torch.tensor([[0.5,0.5],[0.3,0.6]]).reshape(2,1,dim)
x_temp_values =[aa[i, :, :] for i in range(aa.size(0))] # List of x_temp values
model = model  # Your model object
D = 2  # D value
fstar = fstar # fstar value

pool = mp.Pool(processes=mp.cpu_count())  # Use the number of available CPUs

# # # Map the function to the x_temp values using multiple processes
results = pool.starmap(logpdf_calculation, [(x, model, D, fstar) for x in x_temp_values])

RuntimeError: Cowardly refusing to serialize non-leaf tensor which requires_grad, since autograd does not support crossing process boundaries.  If you just want to transfer the data, call detach() on the tensor before serializing (e.g., putting it on the queue).